<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1:
- Nombre de alumno 2:


### **Link de repositorio de GitHub:** `http://....`

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Obtener datos desde Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Optimizar a través de librerías funciones.

#Importamos librerias utiles 😸

In [39]:
%%capture
!pip install praw
!pip install line_profiler
!pip install memory_profiler
!pip install numba

import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

import IPython.core
IPython.core.page = print

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://styles.redditmedia.com/t5_2rer8/styles/bannerBackgroundImage_xzrb6my122181.jpg">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada liena de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [ ] Estudiar la memoria ocupada por el código a través de un perfilador.
- [ ] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

In [64]:
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [65]:
votes, post, url = praw_reddit()

Se usan las librerias `line_profiler` y `memory_profiler` tál y como se muestra en el [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html)

In [66]:
%%file reddit.py
import praw
import numpy as np

def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

Overwriting reddit.py


In [67]:
%reload_ext line_profiler
%reload_ext memory_profiler

In [68]:
%lprun -f praw_reddit praw_reddit()

```
Timer unit: 1e-06 s

Total time: 21.3502 s
File: /tmp/ipykernel_47299/3968796995.py
Function: praw_reddit at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def praw_reddit(nombre_subreddit="chile", n_hot=1000):
     2         2        943.0    471.5      0.0      reddit = praw.Reddit(
     3         1          3.0      3.0      0.0          client_id="-w2hyFINxZ8T3g",
     4         1          2.0      2.0      0.0          client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
     5         1          2.0      2.0      0.0          password="ClasesMDS7202",
     6         1          2.0      2.0      0.0          user_agent="Clases",
     7         1          1.0      1.0      0.0          username="DocenciaDataScience",
     8         1          1.0      1.0      0.0          check_for_async=False,
     9                                               )
    10         1         49.0     49.0      0.0      subreddit = reddit.subreddit(nombre_subreddit)
    11                                           
    12         1          3.0      3.0      0.0      votes, post, url = {}, {}, {}
    13         1   21196282.0 21196282.0     99.3      top_submissions = list(subreddit.hot(limit=n_hot))
    14        20         60.0      3.0      0.0      for it, top_n in enumerate(range(50, len(top_submissions), 50)):
    15        19        197.0     10.4      0.0          top_n_submissions = top_submissions[:top_n]
    16        19        148.0      7.8      0.0          upvotes, downvotes, url[it], post[it] = [], [], [], []
    17                                           
    18      9519      11964.0      1.3      0.1          for submission in top_n_submissions:
    19      9500      10995.0      1.2      0.1              try:
    20      9500      14786.0      1.6      0.1                  ratio = submission.upvote_ratio
    21     19000      24100.0      1.3      0.1                  ups = int(
    22      9500      17183.0      1.8      0.1                      round((ratio * submission.score) / (2 * ratio - 1))
    23      9500      11363.0      1.2      0.1                      if ratio != 0.5
    24        88        167.0      1.9      0.0                      else round(submission.score / 2)
    25                                                           )
    26      9500      13141.0      1.4      0.1                  upvotes.append(ups)
    27      9500      14436.0      1.5      0.1                  downvotes.append(ups - submission.score)
    28      9500      16095.0      1.7      0.1                  post[it].append(submission.title)
    29      9500      15856.0      1.7      0.1                  url[it].append(submission.url)
    30                                                       except Exception as e:
    31                                                           continue
    32        19       2392.0    125.9      0.0          votes[it] = np.array([upvotes, downvotes]).T
    33         1          2.0      2.0      0.0      return votes, post, url
```

In [69]:
from reddit import praw_reddit

%mprun -f praw_reddit praw_reddit()

```
Filename: /home/step/Code/MDS7202-2021/lab09/reddit.py

Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
     4    177.3 MiB    177.3 MiB           1   def praw_reddit(nombre_subreddit="chile", n_hot=1000):
     5    177.3 MiB      0.0 MiB           2       reddit = praw.Reddit(
     6    177.3 MiB      0.0 MiB           1           client_id="-w2hyFINxZ8T3g",
     7    177.3 MiB      0.0 MiB           1           client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
     8    177.3 MiB      0.0 MiB           1           password="ClasesMDS7202",
     9    177.3 MiB      0.0 MiB           1           user_agent="Clases",
    10    177.3 MiB      0.0 MiB           1           username="DocenciaDataScience",
    11    177.3 MiB      0.0 MiB           1           check_for_async=False,
    12                                             )
    13    177.3 MiB      0.0 MiB           1       subreddit = reddit.subreddit(nombre_subreddit)
    14                                         
    15    177.3 MiB      0.0 MiB           1       votes, post, url = {}, {}, {}
    16    177.3 MiB      0.0 MiB           1       top_submissions = list(subreddit.hot(limit=n_hot))
    17    177.3 MiB      0.0 MiB          20       for it, top_n in enumerate(range(50, len(top_submissions), 50)):
    18    177.3 MiB      0.0 MiB          19           top_n_submissions = top_submissions[:top_n]
    19    177.3 MiB      0.0 MiB          19           upvotes, downvotes, url[it], post[it] = [], [], [], []
    20                                         
    21    177.3 MiB      0.0 MiB        9519           for submission in top_n_submissions:
    22    177.3 MiB      0.0 MiB        9500               try:
    23    177.3 MiB      0.0 MiB        9500                   ratio = submission.upvote_ratio
    24    177.3 MiB      0.0 MiB       19000                   ups = int(
    25    177.3 MiB      0.0 MiB        9500                       round((ratio * submission.score) / (2 * ratio - 1))
    26    177.3 MiB      0.0 MiB        9500                       if ratio != 0.5
    27    177.3 MiB      0.0 MiB          71                       else round(submission.score / 2)
    28                                                         )
    29    177.3 MiB      0.0 MiB        9500                   upvotes.append(ups)
    30    177.3 MiB      0.0 MiB        9500                   downvotes.append(ups - submission.score)
    31    177.3 MiB      0.0 MiB        9500                   post[it].append(submission.title)
    32    177.3 MiB      0.0 MiB        9500                   url[it].append(submission.url)
    33                                                     except Exception as e:
    34                                                         continue
    35    177.3 MiB      0.0 MiB          19           votes[it] = np.array([upvotes, downvotes]).T
    36    177.3 MiB      0.0 MiB           1       return votes, post, url
```

**Respuesta:**

* Se observa que lo que toma más tiempo en este caso es la llamada a `list(subreddit.hot(limit=n_hot))`. Esto se debe realizar una request a la API de Reddit. Es decir, en este caso el mayor *bottleneck* se debe a la velocidad de internet.

## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder las preguntas.
- [ ] Mejorar el código con cache.
- [ ] Explicar las consecuencias de usar lru_cache sobre praw_reddit

**Respuestas Teóricas:**

* La memoria cache es un tipo de memoria en el cuál se almacenan resultados o valores de ejecuciones previas para ser reutilizadas en futuras ejecuciones. Un ejemplo de este tipo de memoria es la memoria cache de una CPU la cuál almacena las instrucciones más usadas. Las siglas LRU corresponden a *Least recently used*. Este es un tipo de algoritmo que determina que elementos se eliminan de la cache, el cuál, en este caso, descarta los elementos más antiguos (el usado menos reciententemente).


* Los costos de usar técnicas de Caching proviene principalmente velocidad de acceso y hardware: la velocidad de acceso a la caché debe ser menor al tiempo que se demoraría ejecutar las instrucciones necesarias para obtener el mismo resultado. Esto implica generalmente costos a nivel de hardware ya que para tener una mayor velocidad de acceso es necesario usar hardware especializado como la memoria RAM o en el caso de la CPU memorias L1, L2, L3... Estos componentes resultan ser más caros que un simple disco duro de almacenamiento.


* Una consecuencia podría ser usar una caché desactualizada. Con esto se refiere a que los datos almacenados ya no son relevantes o que ya no corresponden a los `top_n` actualmente. Es decir, entre el tiempo que se almacenaron los resultados y que se volvió a ejecutar la función las `top_n_submissions` ya no son las mismas.

In [70]:
###### Código Aquí ######
from functools import lru_cache
import praw
import numpy as np

@lru_cache
def praw_reddit_cache(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [71]:
%%file reddit_cache.py
from functools import lru_cache

import praw
import numpy as np

@lru_cache
def praw_reddit_cache(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

Writing reddit_cache.py


In [89]:
%timeit praw_reddit()

21.7 s ± 1.06 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [90]:
%timeit praw_reddit_cache()

79.7 ns ± 10.2 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


**Respuesta de lru_cache sobre praw_reddit**

`Respondido anteriormente`

## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [95]:
###### Código Aquí ######
import math

import numpy as np

def p_mean_and_std(votes):
    mus, sigmas = [], []
    for vote in votes:
        a = vote[0] + 1
        b = vote[1] + 1

        sigma = 1.65 * (1 / (a+b)) * math.sqrt(a*b/((a + b + 1)))
        mu = a / (a + b)
        mus.append(mu)
        sigmas.append(sigma)

    return mus, sigmas

def p_mean_and_std_np(votes):
    votes = votes + 1
    a = votes[:, 0]
    b = votes[:, 1]

    sigma = 1.65 * (1 / (a+b) * np.sqrt(a*b/(a + b + 1)))
    mu = a / (a + b)

    return mu, sigma

## 1.4 Comparación de Rendimiento [2 Puntos]

Creadas las funciones de la sección 1.3, de forma iterativa compare los diferentes batches de datos que contiene el diccionario `votes`. Para esto genere un gráfico utilizando plotly, donde se pueda observar las diferencias de tiempo que toma ejecutar las diferentes cantidades de datos. ¿es posible observar una diferencia? ¿a qué se deberá esto?.

Aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. 

In [101]:
import time

import pandas as pd
import plotly.express as px

native_res = {}
numpy_res = {}

for key, batch in votes.items():
    t0 = time.perf_counter_ns()
    p_mean_and_std(batch)
    t1 = time.perf_counter_ns()
    native_res[key] = (t1 - t0) / 1000

    t0 = time.perf_counter_ns()
    p_mean_and_std_np(batch)
    t1 = time.perf_counter_ns()
    numpy_res[key] = (t1 - t0) / 1000

df = pd.DataFrame.from_dict({
    "batches": native_res.keys(), 
    "Nativo": native_res.values(), 
    "Numpy": numpy_res.values() 
})

fig = px.line(df, x='batches', y=["Nativo", "Numpy"], title="Comparación de tiempos de ejecución en microsegundos")
fig.show()

Ahora, grafiquen los tiempos tomados y comente los desempeños.

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

In [102]:
###### Código Aquí ######
@jit
def p_mean_and_std(votes):
    mus, sigmas = [], []
    for vote in votes:
        a = vote[0] + 1
        b = vote[1] + 1

        sigma = 1.65 * (1 / (a+b)) * math.sqrt(a*b/((a + b + 1)))
        mu = a / (a + b)
        mus.append(mu)
        sigmas.append(sigma)

    return mus, sigmas

@jit(nopython=True)
def p_mean_and_std_np(votes):
    votes = votes + 1
    a = votes[:, 0]
    b = votes[:, 1]

    sigma = 1.65 * (1 / (a+b) * np.sqrt(a*b/(a + b + 1)))
    mu = a / (a + b)

    return mu, sigma

In [103]:
###### Código Aquí ######
native_res = {}
numpy_res = {}

p_mean_and_std(votes[0])
p_mean_and_std_optimized(votes[0])

for key, batch in votes.items():
    t0 = time.perf_counter_ns()
    p_mean_and_std(batch)
    t1 = time.perf_counter_ns()
    native_res[key] = (t1 - t0) / 1000

    t0 = time.perf_counter_ns()
    p_mean_and_std_np(batch)
    t1 = time.perf_counter_ns()
    numpy_res[key] = (t1 - t0) / 1000

df = pd.DataFrame.from_dict({
    "batches": native_res.keys(), 
    "Nativo con Numba": native_res.values(), 
    "Numpy con Numba": numpy_res.values() 
})

fig = px.line(
    df, 
    x='batches', 
    y=["Nativo con Numba", "Numpy con Numba"], 
    title="Comparación de tiempos de ejecución en microsegundos"
)
fig.show()

## 1.5 Plot de Resultados [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [106]:
###### Código Aquí ######

# No habia nada unu

**Respuesta:**

`Escriba su respuesta aquí`

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>